In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('D:/Canada/University of Waterloo/3. Spring 2020/ECE 657/Assignments/Assignment 1/train_data.csv', header = None)
label = pd.read_csv('D:/Canada/University of Waterloo/3. Spring 2020/ECE 657/Assignments/Assignment 1/train_labels.csv', header = None)

In [3]:
train = np.array(train)
label = np.array(label)

In [4]:
#Combined the train data and label to split into training and validation data
combined = np.c_[train,label]

 - Train and Validation Split
##### Features and labels are combined to split the dataset into training set and validation set. I have splitted the data set into two parts 90% and 10% for training and validation respectively. I prefered 90% and 10% considering large amount of data points i.e 24752 which will result in 22278 data points for training and 2476 datapoints for validation.

In [5]:
# Splitting the data and labels for training dataset
X_train = combined[:int(combined.shape[0]*0.9),:784]
y_train = combined[:int(combined.shape[0]*0.9),784:]

In [6]:
#Splitting the data and labels for validation dataset
X_val = combined[int(combined.shape[0]*0.9):,:784]
y_val = combined[int(combined.shape[0]*0.9):,784:]

In [7]:
#Checking the shape of the training and Validation data
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(22278, 784)
(22278, 4)
(2476, 784)
(2476, 4)


### Details about the Neural Network
- Architecture
##### My neural network consists of 784 input perceptron, 44 perceptron in hidden layer and 4 perceptrons in the output layer. The reason for selecting small number of neuron is with the intuation of having image dataset observing values between 0-255 in middle features. The patterns required to classify the images with lots of 0 are less resuting in small number of neurons. The exact number of neuron was obtained fine tunning the model based on the validation accuracy.</br>


 - Activation Function
##### The activation function used in the hidden layer is the sigmoid and for output layer softmax is used considering multi class one hot encoded labels.</br>


 - Loss Function
##### The loss function used for this multi label class problem is cross entropy as it penalizes for predicting the wrong class and always tries to minimize the distance between the actual and predicted labels.</br>


 - Back Propagation
##### Mini Batch back propagation is implemented instead of batch backpropagation because of the faster convergence to the global monima compared to the batch backpropagation. The back propagation is carried out (Total No. of samples/Batch size) i.e 2785 time in one epoch for mini batch back propagation.</br>


 - Accuracy
##### The Training accuracy achived is 99.94% and validation accuracy is 99.03%.</br>

In [8]:
class NeuralNetwork():
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.Nl = 2
        self.dim = [784, 44, 4]
        self.loss = []
        self.param = {}
        self.inter = {}
        self.sample = self.y.shape[0]
        self.lr = 0.75

# Initializing the weights and bias of the requied dimensions randomly
    def weightinit(self):
        np.random.seed(42)
        self.param['W1'] = np.random.randn(self.dim[0], self.dim[1])
        self.param['b1'] = np.zeros((1, self.dim[1]))
        self.param['W2'] = np.random.randn(self.dim[1], self.dim[2])
        self.param['b2'] = np.zeros((1, self.dim[2]))
        return

#Sigmoid is the activation function used in the hidden layer
    def sigmoid(self, z):
        sig = 1 / (1 + np.exp(-z))
        return sig

#Softmax is the function used in the output layer to get the probability for each of class
#Softmax is used because of the multi class classification problem
    def softmax(self, z):
        exps = np.exp(z)
        return exps / np.sum(exps, axis=1, keepdims=True)

#One sided cross entropy function is used for the multi class classification problem
#As we have 4 classes : 0, 1,2 and 3
    def lossfunction(self, predict, actual):
        n_samples = actual.shape[0]
        true_class = actual.argmax(axis=1)
        logrithm = - np.log(predict[np.arange(n_samples), true_class])
        cross = np.sum(logrithm) / n_samples
        return cross

#Forward function is used for the matrix calculations of forward propagation and for prediction of the class
#Input is the N_sample X features matrix
#Output is Predicted one hot encoded numpy array and loss
    def Forward(self):
        #Matrix multiplication of W1 and input features 
        z1 = np.dot(self.X, self.param['W1']) + self.param['b1']
        #Passing it through sigmoid generates the Output for each of the perceptron in hidden layer
        a1 = self.sigmoid(z1)
        #Matrix multiplication of a1 and W2
        z2 = np.dot(a1, self.param['W2']) + self.param['b2']
        #Passing it through softmax generates the probability for each of the class
        a2 = self.softmax(z2)
        self.inter['a1'] = a1
        self.inter['z1'] = z1
        self.inter['a2'] = a2
        self.inter['z2'] = z2
        self.predict = a2
        return self.predict

#sigmoid_deriv gives the derivation of the sigmoid function
    def sigmoid_deriv(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))

#Backprop function is to implement the back propogation after calculating the error
#Based on the dE/dw for every weight, the weights are updated
    def Backprop(self):
        m = self.X.shape[0]
        #loss calculation 
        loss = self.lossfunction(self.predict, self.y)
        #dz2 = dE/da2*da2/dz2 which is equal to (prediction - Real) for cross entropy and softmax function
        dz2 = (self.predict - self.y) / m
        da1 = np.dot(dz2, self.param['W2'].T)
        dz1 = da1 * self.sigmoid_deriv(self.inter['z1'])

        #Updating the weights and bias term based on the dE/dw
        self.param['W2'] = self.param['W2'] - self.lr * np.dot(self.inter['a1'].T, dz2)
        self.param['b2'] = self.param['b2'] - self.lr * np.sum(dz2, axis=0, keepdims=True)
        self.param['W1'] = self.param['W1'] - self.lr * np.dot(self.X.T, dz1)
        self.param['b1'] = self.param['b1'] - self.lr * np.sum(dz1, axis=0, keepdims=True)
        return loss

#prediction function is called after training the model for required number of epochs
#It calls the Forward propogation to predict the one hot encoded labels for given inputs
#Input is the data points to be predicted and returns the one hot encoded in numpy array
    def prediction(self, data):
        self.X = data
        pred = self.Forward()
        pred = np.where(pred > 0.5, 1, 0)
        return pred

#This function is used to store the weights of the trained model as separate file .npy file
    def Weightstore(self):
        self.weight = []
        for key in self.param.keys():
            self.weight.append(self.param[key])
        np.save('weights.npy', self.weight)
        return

#This function enables to implement the mini batch size backpropogation
#Rather than feeding whole training data once, input of some batch sizes are fed and then backpopogated to the update weights
#This function is being called (totaldata/batchsize) times in 1 Epoch resulting in the weight update for same time in 1 Epoch
#Input is batch sized input and fits the model
    def mini_batch_implementation(self, data, label):
        self.X = data
        self.y = label
        pred = self.Forward()
        loss = self.Backprop()
        return pred, loss

In [9]:
#Defining Batch size and number of epochs for which model is required to train
batch_size = 8
epochs = 15
#Calling the constructor of the NeuralNetwork class
nn = NeuralNetwork(X_train[0:batch_size], np.array(y_train[0:batch_size]))
#Initializing the weights
nn.weightinit()

In [10]:
# These two for loops fits the model in batches of input of size 8 and prints the loss value every Epoch
for x in range(epochs):
    for a in range(0,len(X_train),batch_size):
        pred, loss = nn.mini_batch_implementation(X_train[a:a+batch_size], np.array(y_train[a:a+batch_size]))
    print("Epoch " + str(x + 1) + " : " + "Training Loss " + str(loss))

Epoch 1 : Training Loss 0.0597843176650113
Epoch 2 : Training Loss 0.0011780151972225607
Epoch 3 : Training Loss 0.0008209587275510438
Epoch 4 : Training Loss 0.00035987401293843406
Epoch 5 : Training Loss 0.0003283729837512418
Epoch 6 : Training Loss 0.00021211293986590646
Epoch 7 : Training Loss 0.0007456454246492077
Epoch 8 : Training Loss 0.000733538122934998
Epoch 9 : Training Loss 0.00041182598860860453
Epoch 10 : Training Loss 0.0016278069054302027
Epoch 11 : Training Loss 0.0015710918945201153
Epoch 12 : Training Loss 0.0008258077595258102
Epoch 13 : Training Loss 0.0004603653012098317
Epoch 14 : Training Loss 0.0013974648452064968
Epoch 15 : Training Loss 0.0012114601083762493


In [11]:
#This class function call stores the weights of the model in the current directory
nn.Weightstore()

In [12]:
#nn.prediction function returns the one hot encoded numpy array
prediction_function = nn.prediction(X_val)

In [13]:
#This function calls the prediction function of the class and return the accuracy in percentage
def get_accuracy(X , y):
    accuracy = 0
    y_hat = nn.prediction(X)
    for i in range(y_hat.shape[0]):
        accuracy += 1 if (y[i] == y_hat[i]).all() else 0
    return (accuracy/y.shape[0])*100

In [14]:
#Testing the Training and Validation accuracy
train_acc = get_accuracy(X_train, np.array(y_train))
val_acc = get_accuracy(X_val, np.array(y_val))

In [15]:
print("Training accuracy : ", train_acc)
print("Validation accuracy : ", val_acc)

Training accuracy :  99.94613520064638
Validation accuracy :  99.03069466882067
